# TITLE: An end-to-end homomorphically encrypted neural network

### 1. Introduction

TBD

### 2. Homomorphic encryption

The objective of using homomorphic encryption is that, given ciphertexts that encrypt to $\pi_i$, $\ldots$, $\pi_t$, a computation can be performed that encrypts $f(\pi_i, \ldots, \pi_t)$ for any function $f$ in a way that no information about $\pi_i$, $\ldots$, $\pi_t$ is decrypted at any point, as first described by (2009, Gentry). Formally, a homomorphic encryption scheme is said to be
secure if no adversary has an advantage in guessing (with better than 50% chance) whether a given ciphertext is an encryption of either one of two equally likely distinct messages. This requires encryption to be randomized so that two different encryptions of the same message do not look the same (2018, Standard).

Several implementations of such schemes have been developed since, including DGHV (Dijk, 2010), BGV (Brakerski, 2011) and BFV (Fan-Vercauteren, 2012), with varying levels of security and efficiency. In 2016, Cheon *et al.* introduced HEANN (a.k.a. CKKS), which is a variant of the BFV scheme that is particularly well-suited for computations on real numbers. This scheme is also apt for neural network evaluation where the linear layers have to involve a lot of matrix vector multiplications, as it allows for approximate arithmetic operations on encrypted data. Therefore, neural networks can be considered ideal lattices (source here) for homomorphic encryption, as they are composed of linear layers and activation functions that can be approximated by polynomials.

Following the methodology adopted by (Gentry, 2011), we derive a scheme from HEANN capable of performing inference on encrypted data when applied to neural networks. The main idea is to encode a message $\pi$ into a plaintext polynomial $m$ and encrypt it using a public key $\text{PK}$ to obtain a ciphertext $\text{Enc}(m, \nu)$. In this scheme, $\nu$ stands for the *noise parameter* attached to each polynomial $m$ such that the noise is less than some threshold $\Nu \gg \nu$, as proposed in (Gentry, 2009). This *noise budget* is a critical parameter in the scheme, as it determines the security level of the encryption and the accuracy of the computations. The noise budget is defined during training, and the scheme is designed to ensure that the noise budget is not exceeded at any point during the computation. 

$(m, \nu) \xleftarrow{\text{Encode}} \pi$ 

After encoding, the plaintext polynomial $m$ is encrypted using the public key $\text{PK}$ to obtain the ciphertext $\text{Enc}(m, \nu)$. We use a *probabilistic encryption* scheme, where the encryption function is randomized, such that the same plaintext polynomial $m$ can be encrypted into different ciphertexts $\text{Enc}(m, \nu)$ for different instances of the model it is being applied to. We also use a *public key* encryption scheme so that the encryption function can be performed by anyone with access to the public key $\text{PK}$, enabling the model to be applied to data encrypted by any party.

$\text{Enc}(m, \nu) \xleftarrow{\text{PK}} (m, \nu)$

Homomorphic operations can then be performed on the encrypted data as it is fed into the neural network. Considering the approximate arithmetic nature of our base scheme, the *noise parameter* is treated as part of error during approximate computations (Cheon *et al.*, 2016). In this sense, consider that the encryption $\mathbf{c}$ of message $m$ by the secret key $sk$ will have a structure of the form $\langle \mathbf{c}, sk \rangle = m + e$, where $e$ must be large enough to mask any significant features of message $m$ while not exceeding the noise budget $\nu$. For a noise budget of $0 \leq \nu \leq \Nu$, a ciphertext $\mathbf{c}$ will be decrypted to the message $m$ with a Gaussian distributed error $e$ such that $|e| \leq \nu$.

$\text{Enc}(f(m, \nu)) \xleftarrow{f(\cdot)} \text{Enc}(m, \nu)$

The encrypted output can then be decrypted using a secret key $\text{SK}$ to obtain the plaintext polynomial $m'$. The decryption function is deterministic, such that the same ciphertext $\text{Enc}(f(m, \nu))$ will always decrypt to the same plaintext polynomial $m'$. The decryption function is also designed to ensure that the noise parameter $\nu$ is removed from the decrypt plaintext polynomial $m'$ during decryption.

$f(m') \xleftarrow{\text{SK}} \text{Enc}(f(m, \nu))$

Finally, the plaintext polynomial $m'$ is decoded to obtain the message $\pi'$.

$\pi' \xleftarrow{\text{Decode}} f(m')$

The original HEANN scheme consists of five algorithms ($KeyGen$, $Enc$, $Dec$, $Add$, $Mult$), while ours consists of three algorithms ($KeyGen$, $Enc$, $Dec$), since the neural network will be performing additions and multiplications internally. The $KeyGen$ algorithm generates the public and secret keys, the $Enc$ algorithm encrypts a plaintext polynomial and the $Dec$ algorithm decrypts a ciphertext. Our algorithms are explained below and are based on the HEANN scheme proposed by Cheon *et al.* (2016) and in line with the (2018, Standard).

- $KeyGen(\lambda) \rightarrow (\text{PK}, \text{SK})$: Generates a public key $\text{PK}$ and a secret key $\text{SK}$, where $\lambda$ is the security parameter.
  - Given the security parameter $\lambda$, choose a $M = M(2^\lambda)$, an integer $h = h(\log_2(\lambda))$, a byte lenght $P = P(\lambda, \nu)$, and an $I$ number of iterations.
  - Choose a byte length  $\text{P}$ for the key, and fix the number of iterations  $\text{I}$ to ensure sufficient computational cost.
  - Set  $\mathcal{KDF} = \text{PBKDF2-HMAC-SHA(M)}$ applying the key derivation function iteratively.
  - Derive the key as: $k \leftarrow \mathcal{KDF}(\lambda, \text{H}, \text{P}, \text{I})$

- $Enc(\text{PK}, m) \rightarrow \text{Enc}(m, \nu)$: Encrypts a plaintext polynomial $m$ using the public key $\text{PK}$ to obtain a ciphertext $\text{Enc}(m, \nu)$.
  - for a $n$-dimensional vector $m = (m_1, \ldots, m_n) = Z_i \in \mathbb{R}$ of Gaussian numbers, compute the vector $m' = (m_1', \ldots, m_n') = Z_i' \in \mathbb{Z}_q$ using the public key $\text{PK}$ to obtain a ciphertext $\text{Enc}(m_i, \nu_i)$.
- $Dec(\text{SK}, \text{Enc}(m, \nu)) \rightarrow m'$: Decrypts a ciphertext $\text{Enc}(m, \nu)$ using the secret key $\text{SK}$ to obtain a plaintext polynomial $m'$.
  - for an input polynomial $m = (m_1, \ldots, m_n) = Z_i \in \mathbb{Z}_q$, compute the corresponding vector $m' = (m_1', \ldots, m_n') = Z_i' \in \mathbb{R}$ using the secret key $\text{SK}$ to obtain the plaintext polynomial $m'$. Return the closest vector to $m'$.

Similar to Cheon *et al.* (2016), we do not use a separate plaintext space from an inserted error, so an output $m' = f(m, \nu)$ is slightly different from the ouput for the original message $m$, but the error is considered an approximate value for approximate computations and eventually corrected during activation functions embedded in the neural network. We also do not use Galois keys, as we are not performing any rotations on the ciphertexts.